In [ ]:
from equilibrator_api import ComponentContribution, Q_
cc = ComponentContribution()
cc.p_h = Q_(7.4)
cc.p_mg = Q_(3)
cc.ionic_strength = Q_("0.25M")
cc.temperature = Q_("293.15K")

In [ ]:
import pandas as pd
from fractions import Fraction

def read_csv_with_fallback(file_path):
    for sep in [",", "\t", ";"]:
        for encoding in ["utf-8", "latin1", "ISO-8859-1"]:
            try:
                df = pd.read_csv(file_path, sep=sep, encoding=encoding)
                return df
            except Exception:
                pass
    raise ValueError(f"Could not read file {file_path}")

df_reactions = read_csv_with_fallback("stoichiometry_labeling.csv")

selected_pathways = ["fluxes_labeling_for_MDF"]
df_reactions = df_reactions[df_reactions["pathway"].isin(selected_pathways)]

df_mapping = read_csv_with_fallback("input_metabolite_ranges_default.csv")
metabolite_mapping = dict(zip(df_mapping["Metabolite Acronym"], df_mapping["keggID"]))

def get_compound_id(compound_name):
    return "kegg:" + metabolite_mapping.get(compound_name, compound_name)

parsed_reactions = {}
free_energy_results = []

for index, row in df_reactions.iterrows():
    reaction_name = row["pathway"]
    coeff_fractions = {}
    
    for col in row.index:
        if col in ["pathway"]:
            continue
        value = row[col]
        if value == 0:
            continue
        frac = Fraction(str(value)).limit_denominator()
        coeff_fractions[col] = frac
    
    
    substrate_parts_parse = []
    product_parts_parse = []
    
    
    substrate_parts_display = []
    product_parts_display = []
    
    for compound, frac in coeff_fractions.items():
        abs_frac = abs(frac)
        compound_id = get_compound_id(compound)
        
        
        parse_coeff = f"{float(abs_frac.numerator / abs_frac.denominator)}"
        term_parse = f"{parse_coeff} {compound_id}"
        
        
        if abs_frac.denominator == 1:
            display_coeff = f"{abs_frac.numerator}"
        else:
            display_coeff = f"{abs_frac.numerator}/{abs_frac.denominator}"
        term_display = f"{display_coeff} {compound}"
        
        if frac < 0:
            substrate_parts_parse.append(term_parse)
            substrate_parts_display.append(term_display)
        else:
            product_parts_parse.append(term_parse)
            product_parts_display.append(term_display)
    
    reaction_string_parse = " + ".join(substrate_parts_parse) + " = " + " + ".join(product_parts_parse)
    reaction_string_display = " + ".join(substrate_parts_display) + " = " + " + ".join(product_parts_display)
    
    parsed_reaction = cc.parse_reaction_formula(reaction_string_parse)
    parsed_reactions[reaction_name] = parsed_reaction
    
    standard_dg_prime = cc.standard_dg_prime(parsed_reaction)
    
    dg_mean = round(standard_dg_prime.value.magnitude, 2)
    dg_uncertainty = round(standard_dg_prime.error.magnitude, 2)
    
    free_energy_results.append({
        "Pathway": reaction_name,
        "dG_st": dg_mean,
        "Uncertainty": dg_uncertainty
    })
    
    print(f"Parsed reaction for {reaction_name}: {reaction_string_display}")
    print(f"dG_st for {reaction_name}: {dg_mean} ± {dg_uncertainty} kJ/mol")

df_free_energy = pd.DataFrame(free_energy_results)
df_free_energy.to_csv("standard_dg_labeling.csv", index=False)

In [ ]:
import equilibrator_custom_functions as eq
from equilibrator_api import Q_ 
import importlib
import math
import matplotlib.pyplot as plt 
import custom_plot_functions as cpf

In [ ]:
#Setting physiological conditions as a Python dictionary
cellular_conditions = {
            "p_h": str(Q_("7.4")),
            "p_mg": str(Q_(3)),
            "ionic_strength": str(Q_('250 mM')),
            "temperature": str(Q_(20+273.15,"K")),
            "kcat_source": "fwd",
}

#Generating Compound Settings with the default eQuilibrator bounds
cs_default_bounds = eq.obtain_compound_settings("input_metabolite_ranges_default", custom_bounds = True)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pathway_names = ['fluxes_labeling_for_MDF']
mdf_results = dict( [ (name, eq.MDF_analysis( name, cs_default_bounds, cellular_conditions, custom_dGs = True, print_results = True, y=True)) for name in pathway_names] )

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 8))

for name, mdf_result in mdf_results.items():
    # Obtain necessary data
    MDF, res1, g1, g2, g3, g4, flx, shp_r, r_id, res2, met, bl, bu, c, shp_c = eq.print_MDF_results_individual(mdf_result)

    # Plot the bar chart
    cpf.plot_bar_chart(MDF, g3, g4, r_id, shp_r, ax=ax)

ax.set_ylim([-50, 30])

ax.tick_params(axis='x', rotation=90)

ax.set_xlabel('Reactions', fontsize=20) 
ax.set_ylabel(r'$\Delta G$ (kJ/mol)', fontsize=20)

rounded_MDF = round(MDF, 2)

ax.set_title('MDF'+' = '+str(rounded_MDF)+' kJ/mol', fontsize=20)     

ax.tick_params(axis='both', labelsize=16) 

legend = ax.legend(loc='upper left', fontsize=14)  

plt.subplots_adjust(right=0.8) 

# Show the figure
plt.tight_layout()
plt.show()

# Save the figure
#fig.savefig('MDF_profile.tif', dpi=100)

In [ ]:
import numpy as np
import pandas as pd

standard_dg_prime_rxns = mdf_result.reaction_df['reaction_id']
optimized_dg_prime_values = mdf_result.reaction_df['optimized_dg_prime']
numeric_values = [round(value.magnitude, 2) for value in optimized_dg_prime_values.values]
list_mets = list(standard_dg_prime_rxns)
values_mets = numeric_values
values_mets = np.array(values_mets)
df = pd.DataFrame({'reactions': list_mets, 'deltaG': values_mets})
df.to_csv('optimized_dGs.csv', sep='\t', index=False)
print(df)

In [ ]:
import numpy as np
import pandas as pd
list_mets = list(mdf_result.compound_ids)
values_mets = [conc.magnitude for conc in mdf_result.compound_df.concentration]
values_mets = np.array(values_mets)

df = pd.DataFrame({'Compound': list_mets, 'Concentration (M)': values_mets})
df.to_csv('compound_concentrations.csv', sep='\t', index=False)
print(df)

In [ ]:
import numpy as np
import pandas as pd

standard_dg_prime_rxns = mdf_result.reaction_df['reaction_id']
standard_dg_prime_values = mdf_result.reaction_df['standard_dg_prime']
numeric_values = [round(value.magnitude, 2) for value in standard_dg_prime_values.values]
list_mets = list(standard_dg_prime_rxns)
values_mets = numeric_values
values_mets = np.array(values_mets)
df = pd.DataFrame({'reactions': list_mets, 'deltaG': values_mets})
df.to_csv('standard_dGs.csv', sep='\t', index=False)
print(df)